### Задача 3
---
Склад пункта реализации станков имеет вместимость 20 единиц. Пополнение склада возможно только первого числа каждого месяца. Станки привозят автотранспортом (1 рейс), причем
стоимость рейса складывается из постоянных затрат 50 денежных единиц (ДЕ) и затрат на доставку каждого станка 10 ДЕ. За один рейс (и, следовательно, в месяц) может быть доставлено
не более 5 станков.
Затраты на хранение станка в течение месяца составляют 10 ДЕ.
Ожидаемый спрос на станки приведен в табл. 1. Отсутствие требуемого количества станков
на складе недопустимо.
В начальный момент на складе находится 4 станков.
1. определить план заказов, минимизирующий стоимость;
1. определить границы изменения стоимости хранения станка, в которых найденный план является оптимальным;
1. определить границы изменения постоянных затрат на совершение рейса, в которых найденный план является оптимальным;
1. определить план заказов, минимизирующий стоимость, при условии, что к концу периода на складе должно остаться 3 станков.

Таблица 1: Ожидаемый спрос на станки по месяцам

|  | 1 | 2 | 3 | 4 | 5 | 6 |
|---|:---:|:---:|:---:|:---:|:---:|:---:|
| Спрос, шт. | 0 | 7 | 6 | 3 | 0 | 2 |

In [1]:
warehouse_capacity = 20 # вместимость склада 0..20 - состояния
start_remainder = 4 # изначальный остаток на складе
end_remainder = 3 # конечный остаток на складе

delivery_limit = 5 # ограничение на количество 0..5 управления
delivery_cost = 50 # постоянные затраты на совершение рейса
unit_delivery_cost = 10 # доставка единицы
unit_storage_cost = 10 # стоимость хранения единицы

expected_demand = [0, 7, 6, 3, 0, 2] # ожидаемый спрос по месяцам stages

In [2]:
class Solver:
    def __init__(self, unit_storage_cost, delivery_cost, end_remainder):
        self.unit_storage_cost = unit_storage_cost
        self.delivery_cost = delivery_cost
        self.end_remainder = end_remainder
        self.cache = [{} for _ in expected_demand]
        
    # Расчет стоимости доставки
    def CalcDeliveryCost(self, count):
        res = 0
        if count != 0:
            res = self.delivery_cost + unit_delivery_cost * count
        return res   
    
    #
    def W(self, month, remainder):
        if month >= len(expected_demand):
            if remainder == self.end_remainder:
                return (0, "")
            else:
                return None
        
        if remainder in self.cache[month]:
            return self.cache[month][remainder];
    
        best = None
    
        cur_storage_cost = remainder * self.unit_storage_cost
        restrict = lambda n: expected_demand[month] <= remainder + n <= warehouse_capacity
        for n in filter(restrict, range(delivery_limit + 1)):
            wi = self.W(month + 1, remainder + n - expected_demand[month])
            if wi is not None:
                cur_cost = cur_storage_cost + self.CalcDeliveryCost(n) + wi[0]
                if best is None or cur_cost < best[0]:
                    best = cur_cost, str(n) + wi[1]
                    
        self.cache[month][remainder] = best      
        
        return best

In [3]:
import time
s = Solver(unit_storage_cost, delivery_cost, 0)
start = time.time()
s.W(0, start_remainder)
print("% s seconds" % (time.time() - start))

0.0 seconds


In [4]:
#1
s = Solver(unit_storage_cost, delivery_cost, 0)
cost, seq = s.W(0, start_remainder)
cost, seq

(420, '045500')

In [5]:
#2
reference = Solver(unit_storage_cost, delivery_cost, 0).W(0, start_remainder)[1]
restrict = lambda i: reference == Solver(i, delivery_cost, 0).W(0, start_remainder)[1]
r = list(filter(restrict, range(300)))
print("bounds: ", min(r), max(r))

bounds:  0 12


In [6]:
#3
reference = Solver(unit_storage_cost, delivery_cost, 0).W(0, start_remainder)[1]
restrict = lambda i: reference == Solver(unit_storage_cost, i, 0).W(0, start_remainder)[1]
r = list(filter(restrict, range(300)))
print("bounds: ", min(r), max(r)) # предела сверху вроде как и нет

bounds:  41 299


In [7]:
#4
s = Solver(unit_storage_cost, delivery_cost, end_remainder)
cost, seq = s.W(0, start_remainder)
cost, seq

(460, '045305')